In [1]:
!pip install selenium
!pip install msedge-selenium-tools
!pip install bs4

  Attempting uninstall: selenium
    Found existing installation: selenium 4.7.2
    Uninstalling selenium-4.7.2:
      Successfully uninstalled selenium-4.7.2
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=c5e785c696c1a9d7fa40a0af6bda087157b0d36fb13ee529cd2d01dd68538b5f
  Stored in directory: c:\users\frank\appdata\local\pip\cache\wheels\73\2b\cb\099980278a0c9a3e57ff1a89875ec07bfa0b6fcbebb9a8cad3
Successfully built bs4


In [3]:
!pip install chromedriver_binary==108.0.5359.71.0

  Created wheel for chromedriver-binary: filename=chromedriver_binary-108.0.5359.71.0-py3-none-any.whl size=6911327 sha256=63cfd340c4e02383754a50f447d199d98a888089a9a91901bd09c33fae8f824b
  Stored in directory: c:\users\frank\appdata\local\pip\cache\wheels\f6\59\22\efc96e2766867abd8f2d1d3dd6be102a39ad6ab1d2e19e9474
Successfully built chromedriver-binary


In [4]:
from selenium import webdriver
import chromedriver_binary

#for MSEdge
from msedge.selenium_tools import Edge, EdgeOptions

import csv

In [5]:
# for chrome

driver = webdriver.Chrome()

# # for Firefox
# driver = webdriver.Firefox()

# # for MSEdge
# options = EdgeOptions()
# options.use_chromium = True
# driver = Edge(options=options)

In [6]:
url = 'https://www.amazon.com/'
driver.get(url)

In [13]:
def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}&crid=MV0KIF6CZWSJ&sprefix={}%2Caps%2C304&ref=nb_sb_noss_1'
    keyword = keyword.replace(' ','+')
    return temp.format(keyword, keyword)

In [14]:
url = my_url('laptop Charger')

url

'https://www.amazon.com/s?k=laptop+Charger&crid=MV0KIF6CZWSJ&sprefix=laptop+Charger%2Caps%2C304&ref=nb_sb_noss_1'

# Generic Amazon Scraper Function

In [30]:
from selenium import webdriver
import chromedriver_binary

#for MSEdge
from msedge.selenium_tools import Edge, EdgeOptions

import csv


def my_url(keyword):
    temp = 'https://www.amazon.com/s?k={}&crid=MV0KIF6CZWSJ&sprefix={}%2Caps%2C304&ref=nb_sb_noss_1'
    keyword = keyword.replace(' ','+')
    url = temp.format(keyword, keyword)
    
    url +='&page{}'
    
    return url

def extract_records(obj):
    atag=obj.h2.a
    description=obj.h2.a.text.strip()
    url='https://www.amazon.com/'+atag.get('href')
    
    try:
        parent = obj.find('span', 'a-price')
        price = parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
    try:
        rate = obj.find('span','a-icon-alt').text
        counts_review = obj.find('span','a-size-base s-underline-text').text
    except AttributeError:
        rate = ' '
        counts_review = ' '
    image = obj.find('img',{'class':'s-image'}).get('src')
    
    result = (description, price, rate, counts_review, url, image)
    return result

def main(keyword):
    driver = webdriver.Chrome()
    
    records = []
    url = my_url(keyword)
    
    for page in range(1,3):
        driver.get(url.format(page))
        from bs4 import BeautifulSoup
        soup = BeautifulSoup(driver.page_source,'html.parser')
        soup_results = soup.find_all('div', {'data-component-type':'s-search-result'})
        
        for item in soup_results:
            record = extract_records(item)
            if record:
                records.append(record)
        
        with open('result.csv','w', newline = '', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow(['Description', 'Price', 'Rate', 'Review Counts', 'URL', 'Image'])
            writer.writerows(records)

In [33]:
main('baby cloth')

In [34]:
import pandas as pd

In [35]:
df = pd.read_csv('result.csv')

In [36]:
df.head()

,Description,Price,Rate,Review Counts,URL,Image
0,Simple Joys by Carter's Unisex Babies' Washclo...,$9.00,4.7 out of 5 stars,"(2,561)",https://www.amazon.com//gp/slredirect/picassoR...,https://m.media-amazon.com/images/I/81iV5-cxHP...
1,Yoofoss Muslin Burp Cloths for Baby 10 Pack 10...,$18.95,4.7 out of 5 stars,"(15,470)",https://www.amazon.com//Muslin-Cloths-Cotton-A...,https://m.media-amazon.com/images/I/51ByeYwSPs...
2,"Orighty Baby Washcloths 24-Pack, Microfiber Co...",$8.99,4.6 out of 5 stars,(849),https://www.amazon.com//Orighty-Washcloths-Mic...,https://m.media-amazon.com/images/I/71yr15w-RT...
3,Bamboo Baby Washcloths - 2 Layer Soft Absorben...,$9.98,4.8 out of 5 stars,"(14,893)",https://www.amazon.com//Organic-Bamboo-Baby-Wa...,https://m.media-amazon.com/images/I/61jrCipyr3...
4,Simple Joys by Carter's Unisex Babies' Washclo...,$9.00,4.7 out of 5 stars,"(2,561)",https://www.amazon.com//Simple-Joys-Carters-Wa...,https://m.media-amazon.com/images/I/81iV5-cxHP...
